# NBA HOF Predictor
## Data Manipulation Process

In [2]:
import pandas as pd

## Hall of Fame Dataset

In [3]:
player = pd.read_csv("data/nba-hof.csv")

player.loc[player["hof"] == True].sort_values("num_seasons")

,player_id,player,birth_year,hof,num_seasons,first_seas,last_seas
826,827,John Thompson,NaN,True,2,1965,1966
166,167,Buddy Jeannette,1917.0,True,3,1948,1950
601,602,Maurice Stokes,NaN,True,3,1956,1958
452,453,Don Barksdale,NaN,True,4,1952,1955
2500,2501,Dražen Petrović,NaN,True,4,1990,1993
...,...,...,...,...,...,...,...
2999,3000,Kobe Bryant,NaN,True,20,1997,2016
1722,1723,Robert Parish,NaN,True,21,1977,1997
1604,1605,Moses Malone,1955.0,True,21,1975,1995
3115,3116,Dirk Nowitzki,NaN,True,21,1999,2019


## Player Accolades

In [4]:
all_star = pd.read_csv("data/nba-all-star.csv")
awards = pd.read_csv("data/nba-awards.csv")
end_season = pd.read_csv("data/nba-end-season-teams.csv")
longevity = pd.read_csv("data/nba-longevity.csv")

In [ ]:
all_star_count = all_star.groupby(["player"]).count().sort_values("season", ascending=False)["team"].reset_index(
    name="num_all_star"
)
all_star_count

,player_id,player,birth_year,hof,num_seasons,first_seas,last_seas
0,1,Al Brightman,NaN,False,1,1947,1947
1,2,Al Lujack,NaN,False,1,1947,1947
2,3,Al Negratti,NaN,False,1,1947,1947
3,4,Angelo Musi,NaN,False,3,1947,1949
4,5,Ariel Maughan,1923.0,False,5,1947,1951
...,...,...,...,...,...,...,...
5306,5307,Yuri Collins,NaN,False,1,2025,2025
5307,5308,Yves Missi,NaN,False,1,2025,2025
5308,5309,Zaccharie Risacher,NaN,False,1,2025,2025
5309,5310,Zach Edey,NaN,False,1,2025,2025


In [17]:
# Giving All Star players Player IDs
as_df = pd.merge(
    all_star_count,
    player,
    on = ["player"],
    how="inner"
)[["player_id", "player", "num_all_star", "num_seasons"]]

In [20]:
award_count = awards.loc[awards["winner"] == True].groupby(
    ["player_id", "player", "award"]
).count()["season"].reset_index(name = "award_count")

award_count_wide = award_count.pivot_table(
    index=["player_id", "player"],
    columns="award",
    values="award_count",
    fill_value=0
).reset_index()

In [19]:
end_season_award = end_season.groupby(["player_id", "player", "type", "number_tm"]).count()["season"].reset_index(
    name = "all_team_count")

end_season_award["team_award"] = end_season_award["type"] + " " + end_season_award["number_tm"]

esa_wide = end_season_award.pivot_table(
    index=["player_id", "player"],
    columns="team_award",
    values="all_team_count",
    fill_value=0
).reset_index()

In [25]:
# Combining all accolade tables

merge1 = pd.merge(
    as_df,
    award_count_wide,
    on=["player_id", "player"],
    how="inner"
)

pd.merge(
    merge1,
    esa_wide,
    on=["player_id", "player"],
    how="inner"
)

,player_id,player,num_all_star,num_seasons,aba mvp,aba roy,clutch_poy,dpoy,mip,nba mvp,...,All-ABA 2nd,All-BAA 1st,All-BAA 2nd,All-Defense 1st,All-Defense 2nd,All-NBA 1st,All-NBA 2nd,All-NBA 3rd,All-Rookie 1st,All-Rookie 2nd
0,1204,Kareem Abdul-Jabbar,19,20,0.0,0.0,0.0,0.0,0.0,6.0,...,0.0,0.0,0.0,5.0,6.0,10.0,5.0,0.0,1.0,0.0
1,3000,Kobe Bryant,18,20,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,9.0,3.0,11.0,2.0,2.0,0.0,1.0
2,1381,Julius Erving,16,16,3.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,5.0,2.0,0.0,1.0,0.0
3,2936,Kevin Garnett,15,21,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,9.0,3.0,4.0,3.0,2.0,0.0,1.0
4,2751,Shaquille O'Neal,15,19,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,3.0,8.0,2.0,4.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,4535,Lauri Markkanen,1,8,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
111,2580,Derrick Coleman,1,15,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0
112,4489,De'Aaron Fox,1,8,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
113,3817,Goran Dragić,1,15,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## Player Stats